# Workshop Notebook 3: Observability Part 1 - Validation Rules

In the previous notebooks you uploaded the models and artifacts, then deployed the models to production through provisioning workspaces and pipelines. Now you're ready to put your feet up! But to keep your models operational, your work's not done once the model is in production. You must continue to monitor the behavior and performance of the model to insure that the model provides value to the business.

In this notebook, you will learn about adding validation rules to pipelines.

## Preliminaries

In the blocks below we will preload some required libraries.

In [1]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework


from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import datetime
import time

# used for unique connection names

import string
import random

## Login to Wallaroo

Retrieve the previous workspace, model versions, and pipelines used in the previous notebook.

In [12]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-10"

workspace = wl.get_workspace(workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

model_name = 'aloha-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'aloha-fraud-detector-validation'

pipeline = wl.build_pipeline(pipeline_name)

# verify the workspace/pipeline/model

display(wl.get_current_workspace())
display(prime_model_version)
display(pipeline)



{'name': 'workshop-workspace-john-10', 'id': 10, 'archived': False, 'created_by': '94016008-4e0e-45bc-b2c6-d6f06236b4f5', 'created_at': '2024-09-03T19:13:53.401269+00:00', 'models': [{'name': 'aloha-prime', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 9, 3, 19, 15, 2, 385210, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 9, 3, 19, 15, 2, 385210, tzinfo=tzutc())}, {'name': 'aloha-challenger', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 9, 3, 19, 30, 23, 858702, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 9, 3, 19, 30, 23, 858702, tzinfo=tzutc())}], 'pipelines': [{'name': 'aloha-fraud-detector', 'create_time': datetime.datetime(2024, 9, 3, 19, 15, 18, 22819, tzinfo=tzutc()), 'definition': '[]'}, {'name': 'aloha-fraud-detector-validation', 'create_time': datetime.datetime(2024, 9, 3, 20, 32, 20, 514556, tzinfo=tzutc()), 'definition': '[]'}]}

Name,aloha-prime
Version,5add567b-4427-4d2a-aeff-15c9005a1cc2
File Name,aloha-cnn-lstm.zip
SHA,fd998cd5e4964bbbb4f8d29d245a8ac67df81b62be767afbceb96a03d1a01520
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-03-Sep 19:15:02
Workspace id,10
Workspace name,workshop-workspace-john-10


name,aloha-fraud-detector-validation
created,2024-09-03 20:32:20.514556+00:00
last_updated,2024-09-03 20:32:20.514556+00:00
deployed,(none)
workspace_id,10
workspace_name,workshop-workspace-john-10
arch,None
accel,None
tags,
versions,58f07988-cba0-4774-8703-d20a87f6a938
steps,


## Deploy the Pipeline

Add the model version as a pipeline step to our pipeline, and deploy the pipeline.  You may want to check the pipeline steps to verify that the right model version is set for the pipeline step.

In [13]:
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,aloha-fraud-detector-validation
created,2024-09-03 20:32:20.514556+00:00
last_updated,2024-09-03 20:32:28.299363+00:00
deployed,True
workspace_id,10
workspace_name,workshop-workspace-john-10
arch,x86
accel,none
tags,
versions,"eb517507-4f95-44ac-8a31-24514e670b00, 58f07988-cba0-4774-8703-d20a87f6a938"
steps,aloha-prime


## Model Validation Rules

A simple way to try to keep your model's behavior up to snuff is to make sure that it receives inputs that it expects, and that its output is something that downstream systems can handle. This can entail specifying rules that document what you expect, and either enforcing these rules (by refusing to make a prediction), or at least logging an alert that the expectations described by your validation rules have been violated. As the developer of the model, the data scientist (along with relevant subject matter experts) will often be the person in the best position to specify appropriate validation rules.

In our house price prediction example, suppose you know that house prices in your market are typically in the range $750,000 to $1.5M dollars. Then you might want to set validation rules on your model pipeline to specify that you expect the model's predictions to also be in that range. Then, if the model predicts a value outside that range, the pipeline will log that one of the validation checks has failed; this allows you to investigate that instance further.

Note that in this specific example, a model prediction outside the specified range may not necessarily be "wrong"; but out-of-range predictions are likely unusual enough that you may want to "sanity-check" the model's behavior in these situations.

Wallaroo provides **validations** to detect anomalous data from inference inputs and outputs.  Validations are added to a Wallaroo pipeline with the `wallaroo.pipeline.add_validations` method.

Adding validations takes the format:

```python
pipeline.add_validations(
    validation_name_01 = polars.col(in|out.{column_name}) EXPRESSION,
    validation_name_02 = polars.col(in|out.{column_name}) EXPRESSION
    ...{additional rules}
)
```

* `validation_name`: The user provided name of the validation.  The names must match Python variable naming requirements.
  * **IMPORTANT NOTE**: Using the name `count` as a validation name **returns a warning**.  Any validation rules named `count` are dropped upon request and an warning returned.
* `polars.col(in|out.{column_name})`: Specifies the **input** or **output** for a specific field aka "column" in an inference result.  Wallaroo inference requests are in the format `in.{field_name}` for **inputs**, and `out.{field_name}` for **outputs**.
  * More than one field can be selected, as long as they follow the rules of the [polars 0.18 Expressions library](https://docs.pola.rs/docs/python/version/0.18/reference/expressions/index.html).
* `EXPRESSION`:  The expression to validate. When the expression returns **True**, that indicates an anomaly detected.

The [`polars` library version 0.18.5](https://docs.pola.rs/docs/python/version/0.18/index.html) is used to create the validation rule.  This is installed by default with the Wallaroo SDK.  This provides a powerful range of comparisons to organizations tracking anomalous data from their ML models.

When validations are added to a pipeline, inference request outputs return the following fields:

| Field | Type | Description |
|---|---|---|
| **anomaly.count** | **Integer** | The total of all validations that returned **True**. |
| **anomaly.{validation name}** | **Bool** | The output of the validation `{validation_name}`. |

When validation returns `True`, **an anomaly is detected**.

For example, adding the validation `fraud` to the following pipeline returns `anomaly.count` of `1` when the validation `fraud` returns `True`.  The validation `fraud` returns `True` when the **output** field **dense_1** at index **0** is greater than 0.9.

```python
sample_pipeline = wallaroo.client.build_pipeline("sample-pipeline")
sample_pipeline.add_model_step(ccfraud_model)

# add the validation
sample_pipeline.add_validations(
    fraud=pl.col("out.dense_1").list.get(0) > 0.9,
    )

# deploy the pipeline
sample_pipeline.deploy()

# sample inference
display(sample_pipeline.infer_from_file("dev_high_fraud.json", data_format='pandas-records'))
```

|&nbsp;|time|in.tensor|out.dense_1|anomaly.count|anomaly.fraud|
|---|---|---|---|---|---|
|0|2024-02-02 16:05:42.152|[1.0678324729, 18.1555563975, -1.6589551058, 5...]|[0.981199]|1|True|

### Model Validation Rules Exercise

Add some simple validation rules to the model pipeline that you created in a previous exercise.

* Add an upper bound or a lower bound to the model predictions
* Try to create predictions that fall both in and out of the specified range
* Look through the inference results to check for detected anomalies.

**HINT 1**: since the purpose of this exercise is try out validation rules, it might be a good idea to take a small data set and make predictions on that data set first, *then* set the validation rules based on those predictions, so that you can see the check failures trigger.

Here's an example:

```python
import polars as pl

sample_pipeline = pipeline.add_validations(
    too_low=pl.col("out.main").list.get(0) < 0.90
)
```

For the inference request, isolate only the inferences that trigger the anomaly.  Here's one way using the DataFrame functions:

```python
# sample infer

results = pipeline.infer_from_file('../data/data-1k.df.json')

results.loc[results['anomaly.too_low'] == True,['time', 'out.main', 'anomaly.too_low', 'anomaly.count']].head(20)
```

In [14]:
import polars as pl

sample_pipeline = pipeline.add_validations(
    too_low=pl.col("out.main").list.get(0) < 0.90
)

pipeline.deploy(deployment_config=deploy_config)

sample_pipeline.steps()

[{'ModelInference': {'models': [{'name': 'aloha-prime', 'version': '5add567b-4427-4d2a-aeff-15c9005a1cc2', 'sha': 'fd998cd5e4964bbbb4f8d29d245a8ac67df81b62be767afbceb96a03d1a01520'}]}},
 {'Check': {'tree': ['{"Alias":[{"BinaryExpr":{"left":{"Function":{"input":[{"Column":"out.main"},{"Literal":{"Int32":0}}],"function":{"ListExpr":"Get"},"options":{"collect_groups":"ApplyFlat","fmt_str":"","input_wildcard_expansion":false,"auto_explode":true,"cast_to_supertypes":false,"allow_rename":false,"pass_name_to_apply":false,"changes_length":false,"check_lengths":true,"allow_group_aware":true}}},"op":"Lt","right":{"Literal":{"Float64":0.9}}}},"too_low"]}']}}]

In [16]:
# sample infer

results = pipeline.infer_from_file('../data/cc_data_1k.df.json')

results.loc[results['anomaly.too_low'] == True,['time', 'out.main', 'anomaly.too_low', 'anomaly.count']].head(20)

,time,out.main,anomaly.too_low,anomaly.count
78,2024-09-03 20:33:28.537,[0.0030344203],True,1
79,2024-09-03 20:33:28.537,[0.8821108],True,1
112,2024-09-03 20:33:28.537,[0.8819211],True,1
116,2024-09-03 20:33:28.537,[0.000116970536],True,1
118,2024-09-03 20:33:28.537,[0.7400181],True,1
157,2024-09-03 20:33:28.537,[0.29399034],True,1
158,2024-09-03 20:33:28.537,[0.7350778],True,1
167,2024-09-03 20:33:28.537,[0.4591721],True,1
168,2024-09-03 20:33:28.537,[0.020594902],True,1
185,2024-09-03 20:33:28.537,[0.8989905],True,1


## Clean Up

At this point, if you are not continuing on to the next notebook, undeploy your pipeline to give the resources back to the environment.

In [17]:
## blank space to undeploy the pipeline

pipeline.undeploy()

name,aloha-fraud-detector-validation
created,2024-09-03 20:32:20.514556+00:00
last_updated,2024-09-03 20:32:41.526069+00:00
deployed,False
workspace_id,10
workspace_name,workshop-workspace-john-10
arch,x86
accel,none
tags,
versions,"1537ed92-bf48-4d4a-8103-ceaad3c04a67, eb517507-4f95-44ac-8a31-24514e670b00, 58f07988-cba0-4774-8703-d20a87f6a938"
steps,aloha-prime


## Congratulations!

In this workshop you have

* Set a validation rule on your house price prediction pipeline.
* Detected model predictions that failed the validation rule.

In the next notebook, you will learn how to monitor the distribution of model outputs for drift away from expected behavior.